# Exploring the Senzing Database

After loading data and running entity resolution, Senzing stores everything in PostgreSQL.
This notebook lets you look under the hood at what Senzing created.

In [1]:
import os
import json
import psycopg2
import pandas as pd
import re
from collections import Counter

conn = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST', 'postgres'),
    port=os.getenv('POSTGRES_PORT', '5432'),
    user=os.getenv('POSTGRES_USER', 'postgres'),
    password=os.getenv('POSTGRES_PASSWORD', 'workshop'),
    database=os.getenv('POSTGRES_DB', 'erkg')
)
print('Connected to PostgreSQL')

Connected to PostgreSQL


## Table Overview

Senzing creates 16 tables. Here's what each one holds and how many rows are in it.

In [2]:
table_descriptions = {
    'dsrc_record':    'Raw source records (the original JSON you loaded)',
    'lib_feat':       'Feature library (extracted names, addresses, DOBs, IDs, etc.)',
    'obs_ent':        'Observed entities (one per source record)',
    'res_ent':        'Resolved entities (after entity resolution)',
    'res_ent_okey':   'Maps source records to resolved entities',
    'res_feat_ekey':  'Maps features to entities',
    'res_feat_stat':  'Feature statistics used for scoring',
    'res_rel_ekey':   'Relationship-to-entity key mapping',
    'res_relate':     'Relationships between resolved entities',
    'sys_cfg':        'Engine configurations',
    'sys_codes_used': 'Feature types and codes in use',
    'sys_eval_queue': 'Records queued for re-evaluation',
    'sys_hw_check':   'Hardware check info',
    'sys_sequence':   'ID sequence counters',
    'sys_status':     'System status',
    'sys_vars':       'System variables'
}

cursor = conn.cursor()

rows = []
for table, desc in table_descriptions.items():
    try:
        cursor.execute(f'SELECT COUNT(*) FROM {table}')
        count = cursor.fetchone()[0]
    except Exception:
        conn.rollback()
        count = 'N/A'
    rows.append({'Table': table, 'Rows': count, 'Description': desc})

df_tables = pd.DataFrame(rows)
df_tables

,Table,Rows,Description
0,dsrc_record,282,Raw source records (the original JSON you loaded)
1,lib_feat,8283,"Feature library (extracted names, addresses, D..."
2,obs_ent,282,Observed entities (one per source record)
3,res_ent,196,Resolved entities (after entity resolution)
4,res_ent_okey,282,Maps source records to resolved entities
5,res_feat_ekey,9800,Maps features to entities
6,res_feat_stat,8282,Feature statistics used for scoring
7,res_rel_ekey,984,Relationship-to-entity key mapping
8,res_relate,492,Relationships between resolved entities
9,sys_cfg,2,Engine configurations


## Source Records (`dsrc_record`)

Every record you loaded is stored here with its original JSON. The `dsrc_id` identifies the data source,
and `ent_src_key` is a hash Senzing uses internally.

In [3]:
df_records = pd.read_sql("""
    SELECT dsrc_id, record_id, ent_src_key, json_data
    FROM dsrc_record
    ORDER BY dsrc_id, record_id
""", conn)

print(f'Total source records: {len(df_records)}')
print(f'\nRecords per data source ID:')
print(df_records.groupby('dsrc_id').size().to_string())
print(f'\nSample records:')
df_records[['dsrc_id', 'record_id']].head(10)

Total source records: 282

Records per data source ID:
dsrc_id
1001    258
1002     24

Sample records:


/tmp/ipykernel_7223/2827891406.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_records = pd.read_sql("""


,dsrc_id,record_id
0,1001,10094521532396971848
1,1001,10165632722354515453
2,1001,10264459789712927869
3,1001,10369029484097831758
4,1001,10390699576067371333
5,1001,10442160967680700142
6,1001,10503886662016933171
7,1001,10534369556549149800
8,1001,10540071752248991376
9,1001,10572239874396124657


## Let's check out one record...

In [4]:
sample = df_records.iloc[0]
print(f'Data Source ID: {sample["dsrc_id"]}')
print(f'Record ID: {sample["record_id"]}')
print(f'\nStored JSON:')
print(json.dumps(json.loads(sample['json_data']), indent=2)[:2000])

Data Source ID: 1001
Record ID: 10094521532396971848

Stored JSON:
{
  "DATA_SOURCE": "OPEN-OWNERSHIP",
  "RECORD_ID": "10094521532396971848",
  "statementDate": "2023-06-18",
  "RECORD_TYPE": "ORGANIZATION",
  "NAMES": [
    {
      "PRIMARY_NAME_ORG": "GOLD WYNN UK HOLDINGS LIMITED"
    }
  ],
  "REGISTRATION_DATE": "2020-03-18",
  "REGISTRATION_COUNTRY": "GB",
  "ADDRESSES": [
    {
      "ADDR_TYPE": "BUSINESS",
      "ADDR_FULL": "C/O Fladgate Llp, 16 Great Queen Street, London, WC2B 5DG",
      "ADDR_COUNTRY": "GB"
    }
  ],
  "IDENTIFIERS": [
    {
      "NATIONAL_ID_NUMBER": "12524623",
      "NATIONAL_ID_TYPE": "GB-COH",
      "NATIONAL_ID_COUNTRY": "GBR"
    }
  ],
  "LINKS": [
    {
      "OpenCorporates": "https://opencorporates.com/companies/gb/12524623"
    },
    {
      "OpenOwnership Register": "https://register.openownership.org/entities/18432059995972240708"
    }
  ],
  "RELATIONSHIPS": [
    {
      "REL_ANCHOR_DOMAIN": "OOR",
      "REL_ANCHOR_KEY": "100945215323

## Resolved Entities (`res_ent`)

This is the output of entity resolution — each row is a unique entity that Senzing identified.
282 source records were collapsed into 196 entities (30.5% reduction).

In [5]:
df_entities = pd.read_sql("""
    SELECT res_ent_id, locking_id, last_touch_dt, ent_state
    FROM res_ent
    ORDER BY res_ent_id
""", conn)

print(f'Total resolved entities: {len(df_entities)}')
df_entities.head(10)

Total resolved entities: 196


/tmp/ipykernel_7223/1224330195.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_entities = pd.read_sql("""


,res_ent_id,locking_id,last_touch_dt,ent_state
0,1,3,0,0
1,2,1,0,0
2,3,1,0,0
3,4,1,0,0
4,5,1,0,0
5,6,2,0,0
6,7,1,0,0
7,8,1,0,0
8,9,1,0,0
9,10,2,0,0


## Record-to-Entity Mapping (`res_ent_okey`)

This table shows which source records map to which resolved entities.
When multiple records map to the same `res_ent_id`, that's entity resolution in action.

In [6]:
df_mapping = pd.read_sql("""
    SELECT *
    FROM res_ent_okey
    ORDER BY res_ent_id
""", conn)

print(f'Total mappings: {len(df_mapping)}')
print(f'\nColumns: {list(df_mapping.columns)}')
df_mapping.head(10)

Total mappings: 282

Columns: ['obs_ent_id', 'res_ent_id', 'errule_id', 'match_key']


/tmp/ipykernel_7223/92529021.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mapping = pd.read_sql("""


,obs_ent_id,res_ent_id,errule_id,match_key
0,1,1,0,None
1,145,1,162,+NAME+ADDRESS+NATIONALITY
2,235,1,180,+NAME+DOB+NATIONALITY
3,2,2,0,None
4,3,3,0,None
5,4,4,0,None
6,5,5,0,None
7,6,6,0,None
8,17,6,108,+NAME+ADDRESS+TAX_ID+NATIONAL_ID+REGISTRATION_...
9,7,7,0,None


## Merged entities

Let's look at the entities that merged multiple records (i.e. the interesteing ones).

In [7]:
entity_counts = df_mapping.groupby('res_ent_id').size().reset_index(name='record_count')
merged = entity_counts[entity_counts['record_count'] > 1].sort_values('record_count', ascending=False)

print(f'Entities with multiple records: {len(merged)} out of {len(entity_counts)}')
print(f'\nTop merged entities:')
merged.head(20)

Entities with multiple records: 67 out of 196

Top merged entities:


,res_ent_id,record_count
88,102,4
89,104,4
56,61,4
70,78,3
92,108,3
101,120,3
105,124,3
43,48,3
42,47,3
40,45,3


## Feature Library (`lib_feat`)

Senzing extracts features (names, addresses, phone numbers, IDs, etc.) from every record
and stores them in a normalized library. These features are what get compared during matching.

In [8]:
df_features = pd.read_sql("""
    SELECT *
    FROM lib_feat
    ORDER BY lib_feat_id
    LIMIT 30
""", conn)

print(f'Columns: {list(df_features.columns)}')
df_features

Columns: ['lib_feat_id', 'ftype_id', 'version', 'feat_hash', 'feat_desc', 'felem_values', 'anonymized']


/tmp/ipykernel_7223/2598557074.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_features = pd.read_sql("""


,lib_feat_id,ftype_id,version,feat_hash,feat_desc,felem_values,anonymized
0,1,1,2,72702780939ED12602261A95064134135DD5EDFC,Abassin BADSHAH,2:Abassin BADSHAH|10:SOUTHWESTASIAN|12:PERSON|...,N
1,2,2,2,BC5A459E1CE57A4354B0550E23797E296B439443,1985-05-12,43:1985-05-12|62:1985|63:5|64:12|71:81205|112:...,N
2,3,5,4,F17E683A6A083025F03B8AFB88164DB4D867C54C,"31 Quernmore Close, Bromley, Kent, United King...","28:UNITED KINGDOM|29:31|72:KRNMR KLS|73L:PRML,...",N
3,4,18,1,22F316394D0DFB3A74D037CD5BB18132AB2DA1E1,APSN|PTX,36:APSN\|PTX|,N
4,5,19,1,14459E8DA577F17BC816E1ADDC48B5B5AAEF9095,31|KRNMR KLS||PRML,36:31\|KRNMR KLS\|\|PRML|,N
5,6,19,1,1BAE6EC9392BEC0B75124D3C34848BC4E226E8C5,31|KRNMR KLS||KNT,36:31\|KRNMR KLS\|\|KNT|,N
6,7,19,1,EC6E93158D778F395392D5BD357CAA9B702228DB,31|KRNMR KLS||BR14EL,36:31\|KRNMR KLS\|\|BR14EL|,N
7,8,20,1,CF0DE6BFB15D25222E05402C02E392991CACB083,OTHER_ID=NK25VYVFZT8VDJGGAXMRTWTJ,36:OTHER_ID=NK25VYVFZT8VDJGGAXMRTWTJ|,N
8,9,52,2,BB3ABBF00127585CA036A61B1136046846425DE6,NK-25vyVFzt8vdJGgAXMRTwTJ OPEN-SANCTIONS,37:NK-25vyVFzt8vdJGgAXMRTwTJ|38:OPEN-SANCTIONS...,N
9,10,55,2,926C04B1CFDE891A24A4A1A7F944E7BC985827E8,gb,28:gb|117:GBR|,N


## Counting features by type

In [9]:
df_feat_counts = pd.read_sql("""
    SELECT ftype_id, COUNT(*) as feature_count
    FROM lib_feat
    GROUP BY ftype_id
    ORDER BY feature_count DESC
""", conn)

print('Features by type ID:')
df_feat_counts

Features by type ID:


/tmp/ipykernel_7223/234722146.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_feat_counts = pd.read_sql("""


,ftype_id,feature_count
0,94,2561
1,96,1770
2,92,1728
3,18,471
4,1,390
5,89,275
6,20,190
7,90,187
8,19,150
9,5,149


## Relationships (`res_relate`)

This is the edges of your knowledge graph. Each row is a relationship between two resolved entities.
The `match_key` shows what features matched (+) or didn't match (-) between them.

In [10]:
df_rels = pd.read_sql("""
    SELECT min_res_ent_id, max_res_ent_id, 
           is_disclosed, is_ambiguous, 
           match_key, match_levels
    FROM res_relate
    ORDER BY min_res_ent_id
""", conn)

print(f'Total relationships: {len(df_rels)}')
print(f'Disclosed relationships: {df_rels["is_disclosed"].sum()}')
print(f'Ambiguous relationships: {df_rels["is_ambiguous"].sum()}')
print(f'\nSample relationships:')
df_rels.head(15)

Total relationships: 492
Disclosed relationships: 233
Ambiguous relationships: 2

Sample relationships:


/tmp/ipykernel_7223/2841567355.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rels = pd.read_sql("""


,min_res_ent_id,max_res_ent_id,is_disclosed,is_ambiguous,match_key,match_levels
0,1,156,0,0,+ADDRESS+NATIONALITY-DOB,3
1,1,155,1,0,+ADDRESS+REL_POINTER(DOMAIN:OOR|MIN:|MAX:SHARE...,"3,11"
2,1,99,1,0,+ADDRESS+REL_POINTER(DOMAIN:OOR|MIN:|MAX:APPOI...,"3,11"
3,1,9,1,0,+ADDRESS+REL_POINTER(DOMAIN:OPEN\-SANCTIONS|MI...,"3,11"
4,1,2,1,0,+ADDRESS+REL_POINTER(DOMAIN:OPEN\-SANCTIONS|MI...,"3,11"
5,2,9,0,0,+ADDRESS+REGISTRATION_COUNTRY,3
6,2,99,0,0,+ADDRESS+REGISTRATION_COUNTRY,3
7,5,8,0,0,+ADDRESS+CITIZENSHIP+NATIONALITY+PLACE_OF_BIRT...,3
8,5,13,1,0,+REL_POINTER(DOMAIN:OPEN\-SANCTIONS|MIN:FAMILY...,11
9,5,12,0,0,+ADDRESS+CITIZENSHIP+NATIONALITY-DOB-GENDER-PA...,3


## Analyzing match keys

The following code will show you which features where matched on (indicated as a "+") in the ER and how often, along with which features were NOT matched on (indicated as a "-").

In [11]:
positive_features = Counter()
negative_features = Counter()

for mk in df_rels['match_key']:
    if mk:
        for feat in re.findall(r'\+([A-Z_]+)', mk):
            positive_features[feat] += 1
        for feat in re.findall(r'-([A-Z_]+)', mk):
            negative_features[feat] += 1

print('Features that MATCHED (+) across related entities:')
for feat, count in positive_features.most_common(15):
    print(f'  +{feat}: {count} relationships')

print(f'\nFeatures that DID NOT match (-) across related entities:')
for feat, count in negative_features.most_common(15):
    print(f'  -{feat}: {count} relationships')

Features that MATCHED (+) across related entities:
  +ADDRESS: 287 relationships
  +REL_POINTER: 233 relationships
  +REGISTRATION_COUNTRY: 174 relationships
  +NATIONALITY: 42 relationships
  +NAME: 24 relationships
  +REGISTRATION_DATE: 12 relationships
  +CITIZENSHIP: 7 relationships
  +LIKELY_NAME: 7 relationships
  +DOB: 6 relationships
  +OTHER_ID: 3 relationships
  +PLACE_OF_BIRTH: 2 relationships

Features that DID NOT match (-) across related entities:
  -NATIONAL_ID: 161 relationships
  -REGISTRATION_DATE: 151 relationships
  -RECORD_TYPE: 83 relationships
  -SANCTIONS: 14 relationships
  -DOB: 11 relationships
  -TAX_ID: 6 relationships
  -GENDER: 3 relationships
  -PASSPORT: 3 relationships
  -ADDRESS: 1 relationships


## Engine Configuration (`sys_cfg`)

Senzing stores its configuration as JSON in this table. This includes all data source definitions,
feature types, matching rules, and scoring thresholds.

In [13]:
df_cfg = pd.read_sql("""
    SELECT config_data_id, config_comments, sys_create_dt
    FROM sys_cfg
    ORDER BY sys_create_dt
""", conn)

print('Configuration history:')
df_cfg

Configuration history:


/tmp/ipykernel_7223/4247072541.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cfg = pd.read_sql("""


,config_data_id,config_comments,sys_create_dt
0,948463713,Created by init-database at 2026-02-16T17:36:0...,2026-02-16 17:36:02.089
1,3285108390,"Added data sources: OPEN-OWNERSHIP, OPEN-SANCT...",2026-02-16 17:36:22.543


## Confirm what data sources are presently in our database

In [14]:
cursor = conn.cursor()
cursor.execute("""
    SELECT config_data FROM sys_cfg 
    ORDER BY sys_create_dt DESC LIMIT 1
""")

config = json.loads(cursor.fetchone()[0])

print('Registered data sources:')
for ds in config.get('G2_CONFIG', {}).get('CFG_DSRC', []):
    print(f"  ID {ds['DSRC_ID']}: {ds['DSRC_CODE']}")

Registered data sources:
  ID 1: TEST
  ID 2: SEARCH
  ID 1001: OPEN-OWNERSHIP
  ID 1002: OPEN-SANCTIONS


## Summary Statistics

In [15]:
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*) FROM dsrc_record')
total_records = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM res_ent')
total_entities = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM res_relate')
total_rels = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM lib_feat')
total_features = cursor.fetchone()[0]

print('Senzing Database Summary')
print('=' * 50)
print(f'Source records loaded:     {total_records:>6,}')
print(f'Resolved entities:        {total_entities:>6,}')
print(f'Records merged:           {total_records - total_entities:>6,}')
print(f'Reduction:                {(total_records - total_entities) / total_records * 100:>5.1f}%')
print(f'Entity relationships:     {total_rels:>6,}')
print(f'Features extracted:       {total_features:>6,}')
print(f'Avg features per record:  {total_features / total_records:>6.1f}')
print('=' * 50)

Senzing Database Summary
Source records loaded:        282
Resolved entities:           196
Records merged:               86
Reduction:                 30.5%
Entity relationships:        492
Features extracted:        8,283
Avg features per record:    29.4


In [16]:
conn.close()
print('Connection closed')

Connection closed
